In [6]:
from langchain_ollama.chat_models import ChatOllama
from langchain.schema import HumanMessage, SystemMessage
from langchain.callbacks.base import BaseCallbackHandler

In [7]:
class WSCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        self.tokens = []

    async def on_llm_new_token(self, token: str, **kwargs):
        self.tokens.append(token)
        print("Received new token: ", token)

    def get_tokens(self):
        return self.tokens

In [11]:
from typing import AsyncGenerator


async def stream_agent(question: str) -> AsyncGenerator[str, None]:
    callback = WSCallbackHandler()
    model = ChatOllama(
        model="qwen3",
        temperature=0.7,
        callbacks=[callback],
        reasoning=False,
    )
    messages = [HumanMessage(content=question)]
    for chunk in model.stream(messages):
        yield chunk.content


In [13]:
from IPython.display import Markdown, display

In [14]:
text = ""
async for chunk in stream_agent("What is your name?"):
    text += chunk
    display(Markdown(text), clear=True)

My name is Qwen, and I am a large-scale language model developed by Alibaba Cloud. I am designed to assist with a wide range of tasks, such as answering questions, writing articles, and engaging in conversations. How can I assist you today?

In [15]:
import os
from langchain.document_loaders import CSVLoader, JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
def get_documents():
    docs = []

    for filename in os.listdir("../base_docs"):
        path = os.path.join("../base_docs", filename)
        if filename.endswith(".csv"):
            doc = CSVLoader(file_path=path)
            docs.extend(doc.load())
        elif filename.endswith(".json"):
            doc = JSONLoader(file_path=path, jq_schema=".", text_content=False)
            docs.extend(doc.load())
    return docs

In [61]:
def get_vectorstore():
    documents = get_documents()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")
    vectorstore = FAISS.from_documents(chunks, embeddings)
    return vectorstore

In [62]:
docs = get_documents()

In [63]:
vectorestore = get_vectorstore()
retriever = vectorestore.as_retriever(search_kwargs={"k": 5})


In [64]:
vectorestore.save_local("faiss_index")

In [72]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")
vectorestore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
retriever = vectorestore.as_retriever(search_kwargs={"k": 1})

In [74]:
context_docs = retriever.get_relevant_documents(query="Cual es el horario de Introducción a la programación")
context_text = "\n\n".join([doc.page_content for doc in context_docs])
context_text, context_docs

('DE PROGRAMACION", "groups": [{"code": "1", "schedule": [{"day": "LU", "start": "1115", "end": "1245", "duration": 2, "room": "652", "teacher": "FLORES VILLARROEL CORINA", "isClass": true}, {"day": "MA", "start": "815", "end": "945", "duration": 2, "room": "625D", "teacher": "FLORES VILLARROEL CORINA", "isClass": true}, {"day": "MI", "start": "815", "end": "945", "duration": 2, "room": "652", "teacher": "FLORES VILLARROEL CORINA", "isClass": true}], "teacher": "FLORES VILLARROEL CORINA"}, {"code": "2", "schedule": [{"day": "LU", "start": "645", "end": "815", "duration": 2, "room": "INFLAB", "teacher": "MANZUR SORIA CARLOS B.", "isClass": true}, {"day": "VI", "start": "645", "end": "815", "duration": 2, "room": "651", "teacher": "MANZUR SORIA CARLOS B.", "isClass": true}, {"day": "SA", "start": "645", "end": "815", "duration": 2, "room": "651", "teacher": "MANZUR SORIA CARLOS B.", "isClass": true}], "teacher": "MANZUR SORIA CARLOS B."}, {"code": "3", "schedule": [{"day": "LU", "start":

In [67]:
context_text

'DE PROGRAMACION", "groups": [{"code": "1", "schedule": [{"day": "LU", "start": "1115", "end": "1245", "duration": 2, "room": "652", "teacher": "FLORES VILLARROEL CORINA", "isClass": true}, {"day": "MA", "start": "815", "end": "945", "duration": 2, "room": "625D", "teacher": "FLORES VILLARROEL CORINA", "isClass": true}, {"day": "MI", "start": "815", "end": "945", "duration": 2, "room": "652", "teacher": "FLORES VILLARROEL CORINA", "isClass": true}], "teacher": "FLORES VILLARROEL CORINA"}, {"code": "2", "schedule": [{"day": "LU", "start": "645", "end": "815", "duration": 2, "room": "INFLAB", "teacher": "MANZUR SORIA CARLOS B.", "isClass": true}, {"day": "VI", "start": "645", "end": "815", "duration": 2, "room": "651", "teacher": "MANZUR SORIA CARLOS B.", "isClass": true}, {"day": "SA", "start": "645", "end": "815", "duration": 2, "room": "651", "teacher": "MANZUR SORIA CARLOS B.", "isClass": true}], "teacher": "MANZUR SORIA CARLOS B."}, {"code": "3", "schedule": [{"day": "LU", "start":\

In [68]:
class Agent:
    def __init__(self):
        embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")
        self.vectorestore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
        self.retriever = vectorestore.as_retriever(search_kwargs={"k": 5})

    async def stream_agent_rag(self, question: str) -> AsyncGenerator[str, None]:
        context_docs = self.retriever.get_relevant_documents(query=question)
        context_text = "\n\n".join([doc.page_content for doc in context_docs])

        callback = WSCallbackHandler()
        model = ChatOllama(
            model="qwen3",
            temperature=0.7,
            callbacks=[callback],
            reasoning=False,
        )
        messages = [
            SystemMessage(
                content=f"Responde a la pregunta basado en el siguiente contexto: {context_text}"
            ),
            HumanMessage(content=question),
        ]
        for chunk in model.stream(messages):
            yield chunk.content


In [69]:
agent = Agent()

In [77]:
text = ""
async for chunk in agent.stream_agent_rag(
    "Dame los horarios de Calculo II y sus grupos asignados"
):
    text += chunk
    display(Markdown(text), clear=True)

Los horarios de **"CALCULO II"** y sus grupos asignados son los siguientes:

### **Materia: CALCULO II**
#### **Grupo 6:**
- **Día:** MI  
  - **Hora:** 9:45 - 11:15  
  - **Aula:** 691D  
  - **Profesor:** TERRAZAS LOBO JUAN  
- **Día:** JU  
  - **Hora:** 14:15 - 15:45  
  - **Aula:** 693A  
  - **Profesor:** TERRAZAS LOBO JUAN  
- **Día:** MI  
  - **Hora:** 14:15 - 15:45  
  - **Aula:** 625C  
  - **Profesor:** SOSA MARZE DAVID SAUL  
  - **Nota:** Esta clase no está asignada (isClass: false)

#### **Grupo 6A:**
- **Día:** LU  
  - **Hora:** 12:45 - 14:15  
  - **Aula:** 693D  
  - **Profesor:** BUSTILLOS VARGAS ALEX ISRRAEL  
- **Día:** MA  
  - **Hora:** 9:45 - 11:15  
  - **Aula:** 692D  
  - **Profesor:** BUSTILLOS VARGAS ALEX ISRRAEL  

### **Profesor Asignado:**
- **Grupo 6:** TERRAZAS LOBO JUAN  
- **Grupo 6A:** BUSTILLOS VARGAS ALEX ISRRAEL  

Espera, parece que se interrumpió la información. Si deseas, puedo continuar con el resto de los grupos de **"CALCULO II"**.